# Visualization for attention

In [1]:
#!pip install bertviz

In [2]:
from bertviz import head_view
from bertviz import model_view
import torch
import os
import dill as pickle

## Recorded chemicals 

In [3]:
with open("attention_strings.pkl", "rb") as f:
    samples = pickle.load(f)
for sample in samples:
    print(sample)

['O', 'C', 'C', '[NH2+]', 'C', 'C', 'O', '.', '[O-]', 'C', '=', 'O', '<pad>', '<pad>']
['O', 'C', 'C', '[NH2+]', 'C', 'C', 'O', '.', '[O-]', 'C', '=', 'O', '<pad>', '<pad>']
['O', 'C', 'C', '[NH2+]', 'C', 'C', 'O', '.', '[O-]', 'C', '=', 'O', '<pad>', '<pad>']
['O', 'C', 'C', '[NH2+]', 'C', 'C', 'O', '.', '[O-]', 'C', '=', 'O', '<pad>', '<pad>']
['O', 'C', 'C', '[NH2+]', 'C', 'C', 'O', '.', '[O-]', 'C', '=', 'O', '<pad>', '<pad>']
['O', 'C', 'C', '[NH2+]', 'C', 'C', 'O', '.', '[O-]', 'C', '=', 'O', '<pad>', '<pad>']
['C', '(', '=', 'O', ')', '[O-]', '.', 'C', '(', 'C', 'C', ')', '[NH3+]', '<pad>']
['C', '(', '=', 'O', ')', '[O-]', '.', 'C', '(', 'C', 'C', ')', '[NH3+]', '<pad>']
['C', '(', '=', 'O', ')', '[O-]', '.', 'C', '(', 'C', 'C', ')', '[NH3+]', '<pad>']
['C', '(', '=', 'O', ')', '[O-]', '.', 'C', '(', 'C', 'C', ')', '[NH3+]', '<pad>']
['C', '(', '=', 'O', ')', '[O-]', '.', 'C', '(', 'C', 'C', ')', '[NH3+]', '<pad>']
['C', '(', '=', 'O', ')', '[O-]', '.', 'C', '(', 'C', 'C', ')',

In [4]:
with open("attention_scores.pkl", "rb") as f:
    attentions = pickle.load(f)
[a.shape for a in attentions]

[torch.Size([16, 8, 19, 64]),
 torch.Size([16, 8, 19, 64]),
 torch.Size([16, 8, 19, 64]),
 torch.Size([16, 8, 19, 64]),
 torch.Size([16, 8, 19, 64]),
 torch.Size([16, 8, 19, 64])]

In [5]:
dim = len(attentions[0][0][0])-5 # -5 due to 5 conditions
single_input = [a[-1:, :, :dim, :dim] for a in attentions]
# ins = [s.reshape((8,57,57)) for s in single_input]
# [i.shape for i in ins]
# type(single_input[0])
# len(single_input[0][0][0])
# [s.shape for s in single_input]

In [6]:
s = ['C', '(', '=', 'O', ')', '[O-]', '.', 'C', '(', 'C', 'C', 'C', ')', '[NH3+]']
len(s)

14

In [7]:
#head_view(single_input, s)

In [8]:
#model_view(single_input, s)

In [9]:
def view_attention(num_sample):
    sample = list(filter(lambda x: x!="<pad>", samples[num_sample]))
    dim = len(sample)
    single_input = [a[num_sample:num_sample+1, :, :dim, :dim] for a in attentions]
    model_view(single_input, sample)
    head_view(single_input, sample)

In [21]:
view_attention(0)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
single_input[5][0, 0]

tensor([[ 0.0951,  0.1198,  0.2649, -0.5614, -1.1730, -0.0533, -0.9750, -0.2356,
         -0.0227,  0.4331, -0.0766,  0.0814, -0.0072, -0.8292],
        [ 0.1022,  0.1285,  0.2839, -0.6007, -1.2523, -0.0557, -1.0422, -0.2518,
         -0.0235,  0.4632, -0.0826,  0.0884, -0.0080, -0.8865],
        [ 0.0916,  0.1144,  0.2540, -0.5376, -1.1185, -0.0486, -0.9312, -0.2257,
         -0.0201,  0.4143, -0.0740,  0.0788, -0.0084, -0.7913],
        [ 0.1063,  0.1342,  0.2966, -0.6287, -1.3108, -0.0588, -1.0901, -0.2632,
         -0.0250,  0.4842, -0.0864,  0.0918, -0.0082, -0.9274],
        [ 0.0900,  0.1146,  0.2516, -0.5327, -1.1125, -0.0495, -0.9236, -0.2231,
         -0.0204,  0.4101, -0.0731,  0.0777, -0.0073, -0.7863],
        [ 0.1012,  0.1270,  0.2813, -0.5966, -1.2437, -0.0554, -1.0343, -0.2501,
         -0.0230,  0.4597, -0.0822,  0.0870, -0.0084, -0.8799],
        [ 0.0867,  0.1102,  0.2445, -0.5186, -1.0821, -0.0507, -0.8991, -0.2151,
         -0.0218,  0.3991, -0.0705,  0.0747, -0.0

In [19]:
for n, sample in enumerate(samples):
    for i, val in enumerate(sample):
        if val == '.':
            period = i
    # Need 0-period to only go to 0-period
    eps = 0
    dim = len(sample)
    attention = [a[:, :, :dim, :dim] for a in attentions]
    
    for layer in range(len(attention)):
        for head in range(len(attention[0][0])):
            if torch.any(attention[layer][n, head, 0:period, period+1:len(sample)] > eps):
                break
            if torch.any(attention[layer][n, head, period+1:len(sample), 0:period] > eps):
                break
            print(n, layer, head)